### Introduction to Machine Translation (MT)

This notebook demonstrates the development of a **Transformer-based Machine Translation (MT) model** for translating Turkish to English. Machine Translation is a key application of Natural Language Processing (NLP), focusing on converting text from one language to another.

### Notebook Overview:
1. **Data Preparation**: Turkish-English sentence pairs are loaded, cleaned, and tokenized for training.
2. **Model Definition**: A Transformer model is implemented with standard components like multi-head attention and positional encodings.
3. **Training**: The model is trained using CrossEntropy loss and the Adam optimizer.
4. **Evaluation**: Performance is measured with metrics such as BLEU and ROUGE to assess translation quality.
5. **Inference**: The trained model is tested on new English sentences for Turkish translation.

This notebook provides a concise yet complete workflow for building and testing a Machine Translation model.


In [76]:
from Transformers_libs import *
from Transformers_libs import Transformer
import torch
import numpy as np

In [77]:
english_file = 'trans_eng.txt' # replace this path with appropriate one
turkish_file = 'trans_tr.txt' # replace this path with appropriate one

# Generated this by filtering Appendix code
START_TOKEN = '<s>'
END_TOKEN = '</s>'
PADDING_TOKEN = ''
turkish_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                     ':', '<', '=', '>', '?', '@',
                     '[', '\\', ']', '^', '_', '`',
                     'a', 'b', 'c', 'ç', 'd', 'e', 'f', 'g', 'ğ', 'h', 'i', 'ı', 'j', 'k', 'l',
                     'm', 'n', 'o', 'ö', 'p', 'q', 'r', 's', 'ş', 't', 'u', 'ü', 'v', 'w', 'x',
                     'y', 'z',
                     '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN ]


In [78]:
len(turkish_vocabulary)

77

In [79]:
index_to_turkish = {k:v for k,v in enumerate(turkish_vocabulary)}
turkish_to_index = {v:k for k,v in enumerate(turkish_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [80]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(turkish_file, 'r') as file:
    turkish_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = -100000
english_sentences = english_sentences[TOTAL_SENTENCES:]
turkish_sentences = turkish_sentences[TOTAL_SENTENCES:]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
turkish_sentences = [sentence.rstrip('\n').lower()  for sentence in turkish_sentences]

In [81]:

import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Turkish: {np.percentile([len(x) for x in turkish_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Turkish: 169.0
97th percentile length English: 178.0


In [82]:
english_sentences[4067]

'the agricultural loan given to farmers should be waived totally.'

In [83]:
turkish_sentences[4067]

'çiftçilere verilen tarım kredisi tamamen feragat edilmelidir.'

In [84]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 10) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(turkish_sentences)):
    turkish_sentence, english_sentence = turkish_sentences[index], english_sentences[index]
    if is_valid_length(turkish_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(turkish_sentence, turkish_vocabulary):
        
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(turkish_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")


turkish_sentences = [turkish_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

Number of sentences: 100000
Number of valid sentences: 91674


In [85]:
len(turkish_sentences), len(english_sentences)

(91674, 91674)

In [86]:
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(turkish_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          turkish_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [87]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(200, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding)

In [88]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, turkish_sentences):
        self.english_sentences = english_sentences
        self.turkish_sentences = turkish_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.turkish_sentences[idx]

In [89]:
!pip install scikit-learn

In [90]:
from sklearn.model_selection import train_test_split

train_eng, test_eng, train_tr, test_tr = train_test_split(
    english_sentences,# İngilizce cümleler
    turkish_sentences,  # Türkçe cümleler
    test_size=0.1,      # Test setinin oranı (%20)
    random_state=42     # Sonuçların tekrar edilebilirliği için sabit bir seed
)

# Train ve test datasetleri oluşturma
train_dataset = TextDataset(train_eng, train_tr)
test_dataset = TextDataset(test_eng, test_tr)

# Çıktı kontrolü
print(f"Train set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")
train_loader = DataLoader(train_dataset, batch_size)
test_loader = DataLoader(test_dataset, batch_size)
iterator = iter(train_loader)

Train set size: 82506
Test set size: 9168


In [91]:
train_eng[:5], train_tr[:5]

(['some schools dont even have toilets.',
  'do not fear it.',
  'it also features a 5mp front-facing camera with flash.',
  'the children had complained of nausea, giddiness and stomach ache.',
  'however, rescue operation is underway.'],
 ['bazı okulların tuvaletleri bile yoktur.',
  'korkma.',
  'ayrıca flaşlı 5mp öne bakan bir kameraya sahiptir.',
  'çocuklar bulantı, sersemlik ve mide ağrısından şikayet etmişlerdi.',
  'ancak, kurtarma operasyonu devam ediyor.'])

In [92]:
from torch import nn

criterion = nn.CrossEntropyLoss(ignore_index=turkish_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [93]:
NEG_INFTY = -1e9

def create_masks(eng_batch, tr_batch, device):
    num_sentences = len(eng_batch)
    
    # Initialize masks on the correct device
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length], True )
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length], False)

    # Fill in padding masks based on batch data
    
    for idx in range(num_sentences):
        eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(tr_batch[idx])
        
        # Create padding indices on the CPU
        eng_chars_to_padding_mask = torch.arange(eng_sentence_length + 1, max_sequence_length)
        kn_chars_to_padding_mask = torch.arange(kn_sentence_length + 1, max_sequence_length)
        
        # Apply padding mask directly on the GPU
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    # Create final attention masks with NEG_INFTY on GPU
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    
    # Ensure all masks are on the correct device
    #encoder_self_attention_mask = encoder_self_attention_mask.to(device)
    #decoder_self_attention_mask = decoder_self_attention_mask.to(device)
    #decoder_cross_attention_mask = decoder_cross_attention_mask.to(device)

    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask


In [94]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [95]:
def compute_loss(predictions, labels, valid_indices, criterion, device):
    loss = criterion(
        predictions.view(-1, predictions.size(-1)).to(device),
        labels.view(-1).to(device)
    ).to(device)
    loss = loss.sum() / valid_indices.sum()
    return loss

def train_step(transformer, batch, device, optimizer, criterion,batch_num):
    eng_batch, kn_batch = batch
    encoder_mask, decoder_self_mask, decoder_cross_mask = create_masks(eng_batch, kn_batch, device)
    optimizer.zero_grad()
    predictions = transformer(
        eng_batch, kn_batch,
        encoder_mask.to(device),
        decoder_self_mask.to(device),
        decoder_cross_mask.to(device),
        enc_start_token=False,
        enc_end_token=False,
        dec_start_token=True,
        dec_end_token=True
    )
    labels = transformer.decoder.sentence_embedding.batch_tokenize(
        kn_batch, start_token=False, end_token=True
    )
    valid_indices = torch.where(labels.view(-1) == turkish_to_index[PADDING_TOKEN], False, True)
    loss = compute_loss(predictions, labels, valid_indices, criterion, device)
    loss.backward()
    optimizer.step()
    return loss.item()


In [96]:
def generate_translation(transformer, eng_sentence, device, index_to_turkish, max_sequence_length, END_TOKEN):
    """
    Generate a translation for a given English sentence using the transformer model.

    Args:
        transformer (nn.Module): The trained transformer model.
        eng_sentence (tuple): Input sentence in English (e.g., ("should we go to the mall?",)).
        device (torch.device): Device to use for inference.
        index_to_turkish (dict): Mapping from token indices to Turkish words.
        max_sequence_length (int): Maximum length of the output sentence.
        END_TOKEN (str): The end token used to terminate generation.

    Returns:
        str: The generated Turkish sentence.
    """
    transformer.eval()
    kn_sentence = ("",)  # Başlangıç Türkçe cümlesi (boş)
    for word_counter in range(max_sequence_length):
        # Maskeleri oluştur
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(
            eng_sentence, kn_sentence, device
        )

        # Tahmin yap
        predictions = transformer(
            eng_sentence,
            kn_sentence,
            encoder_self_attention_mask.to(device),
            decoder_self_attention_mask.to(device),
            decoder_cross_attention_mask.to(device),
            enc_start_token=False,
            enc_end_token=False,
            dec_start_token=True,
            dec_end_token=False,
        )

        # Sıradaki token'ı seç
        next_token_prob_distribution = predictions[0][word_counter]
        next_token_index = torch.argmax(next_token_prob_distribution).item()
        next_token = index_to_turkish[next_token_index]

        # Çıktıya ekle
        kn_sentence = (kn_sentence[0] + next_token,)

        # Eğer END_TOKEN bulunursa çeviriyi sonlandır
        if next_token == END_TOKEN:
            break

    return kn_sentence[0]  # Tamamlanmış çeviriyi döndür


In [97]:
print(max_sequence_length)

200


In [98]:
import logging
from torch.utils.tensorboard import SummaryWriter

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# File handler for logging
file_handler = logging.FileHandler('training_logs.txt')
file_handler.setLevel(logging.INFO)

# Console handler for logging
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Set format for log messages
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# Initialize the TensorBoard writer
writer = SummaryWriter(log_dir="runs/transformer_training")

transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 40
for epoch in range(num_epochs):
    logging.info(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        loss = train_step(transformer, batch, device, optim, criterion, batch_num)
        total_loss += loss
        # Append loss value to train losses (if needed)
        if batch_num % 100 == 0:
            eng_batch, kn_batch = batch
            print(eng_batch[0])
            # Generate Turkish translations from the English batch
            kn_sentence_batch = generate_translation(
                transformer,
                eng_sentence=(eng_batch[0],),
                device=device,
                index_to_turkish=index_to_turkish,
                max_sequence_length=max_sequence_length,
                END_TOKEN=END_TOKEN
            )
            logging.info(f"English: {eng_batch[0]}")
            logging.info(f"Turkish Translation: {kn_batch[0]}")
            logging.info(f"Turkish Translation Predict: {kn_sentence_batch}")
            logging.info(f"Iteration {batch_num} : {loss}")

            # Generate evaluation translation for a sample sentence
            kn_sentence_predicted = generate_translation(
                transformer,
                eng_sentence=("should we go to the mall?",),
                device=device,
                index_to_turkish=index_to_turkish,
                max_sequence_length=max_sequence_length,
                END_TOKEN=END_TOKEN
            )

            logging.info(f"Evaluation translation (should we go to the mall?) : {kn_sentence_predicted}")
            logging.info("-------------------------------------------")
            writer.add_scalar('Loss/train', loss, epoch * len(train_loader) + batch_num)

    # Write epoch-based loss to TensorBoard
    writer.add_scalar('Loss/epoch', total_loss / len(train_loader), epoch)

writer.close()


2024-12-01 22:58:01,174 - INFO - Epoch 0
2024-12-01 22:58:01,174 - INFO - Epoch 0
2024-12-01 22:58:01,174 - INFO - Epoch 0
2024-12-01 22:58:01,174 - INFO - Epoch 0


some schools dont even have toilets.


2024-12-01 22:58:01,675 - INFO - English: some schools dont even have toilets.
2024-12-01 22:58:01,675 - INFO - English: some schools dont even have toilets.
2024-12-01 22:58:01,675 - INFO - English: some schools dont even have toilets.
2024-12-01 22:58:01,675 - INFO - English: some schools dont even have toilets.
2024-12-01 22:58:01,678 - INFO - Turkish Translation: bazı okulların tuvaletleri bile yoktur.
2024-12-01 22:58:01,678 - INFO - Turkish Translation: bazı okulların tuvaletleri bile yoktur.
2024-12-01 22:58:01,678 - INFO - Turkish Translation: bazı okulların tuvaletleri bile yoktur.
2024-12-01 22:58:01,678 - INFO - Turkish Translation: bazı okulların tuvaletleri bile yoktur.
2024-12-01 22:58:01,678 - INFO - Turkish Translation Predict: aaa eoaaaaaaaeeeeeeaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaeeeaaaaae    eeeeer` o   s````````raaaaa`````        oorr  aaaaaaaaarrkkkk    oooooooooo```nn   aaaaaammaaaaaaarrrrrrrrraammmmm```rrrrrarrr%
2024-12-01 22:58:01,678 - INFO - Turkish Tran

why should we accept it?


2024-12-01 22:58:07,333 - INFO - English: it had no impact: 7%
2024-12-01 22:58:07,333 - INFO - English: it had no impact: 7%
2024-12-01 22:58:07,333 - INFO - English: it had no impact: 7%
2024-12-01 22:58:07,333 - INFO - English: it had no impact: 7%
2024-12-01 22:58:07,334 - INFO - Turkish Translation: etkisi yoktu:% 7
2024-12-01 22:58:07,334 - INFO - Turkish Translation: etkisi yoktu:% 7
2024-12-01 22:58:07,334 - INFO - Turkish Translation: etkisi yoktu:% 7
2024-12-01 22:58:07,334 - INFO - Turkish Translation: etkisi yoktu:% 7
2024-12-01 22:58:07,335 - INFO - Turkish Translation Predict: buu    e e e e i.</s>
2024-12-01 22:58:07,335 - INFO - Turkish Translation Predict: buu    e e e e i.</s>
2024-12-01 22:58:07,335 - INFO - Turkish Translation Predict: buu    e e e e i.</s>
2024-12-01 22:58:07,335 - INFO - Turkish Translation Predict: buu    e e e e i.</s>
2024-12-01 22:58:07,335 - INFO - Iteration 200 : 3.034865617752075
2024-12-01 22:58:07,335 - INFO - Iteration 200 : 3.0348656177

...

treatment to the patients.


2024-12-01 23:47:14,166 - INFO - English: go to the official website of income tax department @https://www.incometaxindiaefiling.gov.in/
2024-12-01 23:47:14,166 - INFO - English: go to the official website of income tax department @https://www.incometaxindiaefiling.gov.in/
2024-12-01 23:47:14,166 - INFO - English: go to the official website of income tax department @https://www.incometaxindiaefiling.gov.in/
2024-12-01 23:47:14,166 - INFO - English: go to the official website of income tax department @https://www.incometaxindiaefiling.gov.in/
2024-12-01 23:47:14,167 - INFO - Turkish Translation: gelir vergisi departmanı'nın resmi web sitesine gidin @https: //www.incometaxindiaefiling.gov.in/
2024-12-01 23:47:14,167 - INFO - Turkish Translation: gelir vergisi departmanı'nın resmi web sitesine gidin @https: //www.incometaxindiaefiling.gov.in/
2024-12-01 23:47:14,167 - INFO - Turkish Translation: gelir vergisi departmanı'nın resmi web sitesine gidin @https: //www.incometaxindiaefiling.gov.

go to the official website of income tax department @https://www.incometaxindiaefiling.gov.in/


2024-12-01 23:47:14,219 - INFO - Evaluation translation (should we go to the mall?) : hepsi tüm olmalıyız?</s>
2024-12-01 23:47:14,219 - INFO - Evaluation translation (should we go to the mall?) : hepsi tüm olmalıyız?</s>
2024-12-01 23:47:14,219 - INFO - Evaluation translation (should we go to the mall?) : hepsi tüm olmalıyız?</s>
2024-12-01 23:47:14,219 - INFO - Evaluation translation (should we go to the mall?) : hepsi tüm olmalıyız?</s>
2024-12-01 23:47:14,220 - INFO - -------------------------------------------
2024-12-01 23:47:14,220 - INFO - -------------------------------------------
2024-12-01 23:47:14,220 - INFO - -------------------------------------------
2024-12-01 23:47:14,220 - INFO - -------------------------------------------
2024-12-01 23:47:16,859 - INFO - English: chief minister siddaramaiah
2024-12-01 23:47:16,859 - INFO - English: chief minister siddaramaiah
2024-12-01 23:47:16,859 - INFO - English: chief minister siddaramaiah
2024-12-01 23:47:16,859 - INFO - Engli

chief minister siddaramaiah


In [99]:
# Sadece model ağırlıklarını kaydetme
torch.save(transformer.state_dict(), "MT_model_weights.pth")

# Tam modeli (mimari + ağırlıklar) kaydetme
torch.save(transformer, "MT_full_model.pth")

# Checkpoint (model + optimizer + diğer bilgileri) kaydetme
torch.save({
    'epoch': 40,
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optim.state_dict(),
    'loss': 0.5,
}, "MT_model_checkpoint.pth")

In [100]:
!tensorboard --logdir=runs


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


## Inference

In [101]:

transformer.eval()
def translate(eng_sentence,device):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence,device)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_turkish[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]


In [102]:

translation = translate("what should we do when the day starts?",device)
print(translation)

günlerin ne zaman başlamalıyız?</s>


In [103]:

translation = translate("how is this the truth?",device)
print(translation)

bu nasıl gerçekli?</s>


In [ ]:
import logging

# Configure logging for inference
inference_logger = logging.getLogger("InferenceLogger")
inference_logger.setLevel(logging.INFO)

# File handler for inference logs
inference_file_handler = logging.FileHandler('inference.txt')
inference_file_handler.setLevel(logging.INFO)

# Set format for log messages
inference_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
inference_file_handler.setFormatter(inference_formatter)

# Add the file handler to the logger
inference_logger.addHandler(inference_file_handler)

# Inference process
iterator = iter(test_loader)
for batch_num, batch in enumerate(iterator):
    transformer.train()
    eng_batch, tr_batch = batch
    for index, eng_sentence in enumerate(eng_batch):
        translation = translate(eng_sentence, device)
        inference_logger.info(f"Sentence: {eng_sentence}")
        inference_logger.info(f"Predict: {translation}")
        inference_logger.info(f"GT: {tr_batch[index]}")
        inference_logger.info("******************")


2024-12-02 00:27:38,551 - INFO - Sentence: perform 15 repetition 3 sets.
2024-12-02 00:27:38,551 - INFO - Sentence: perform 15 repetition 3 sets.
2024-12-02 00:27:38,551 - INFO - Sentence: perform 15 repetition 3 sets.
2024-12-02 00:27:38,551 - INFO - Sentence: perform 15 repetition 3 sets.
2024-12-02 00:27:38,551 - INFO - Predict: perf'den 3 sandalyesi teklif 15 tekrarlar.</s>
2024-12-02 00:27:38,551 - INFO - Predict: perf'den 3 sandalyesi teklif 15 tekrarlar.</s>
2024-12-02 00:27:38,551 - INFO - Predict: perf'den 3 sandalyesi teklif 15 tekrarlar.</s>
2024-12-02 00:27:38,551 - INFO - Predict: perf'den 3 sandalyesi teklif 15 tekrarlar.</s>
2024-12-02 00:27:38,552 - INFO - GT: 15 tekrar 3 set yapın.
2024-12-02 00:27:38,552 - INFO - GT: 15 tekrar 3 set yapın.
2024-12-02 00:27:38,552 - INFO - GT: 15 tekrar 3 set yapın.
2024-12-02 00:27:38,552 - INFO - GT: 15 tekrar 3 set yapın.
2024-12-02 00:27:38,552 - INFO - ******************
2024-12-02 00:27:38,552 - INFO - ******************
2024-12-

## Evalation 

### Evaluation Metrics for Machine Translation

To assess the performance of the machine translation model, the following metrics are utilized:

1. **BLEU Score**:
   - BLEU (Bilingual Evaluation Understudy) measures the overlap between the predicted translations and the reference translations. It focuses on n-gram precision while considering brevity penalties for overly short predictions.
   - The average BLEU score reflects how closely the model's outputs match human translations in terms of word and phrase choices.


2. **ROUGE Scores**:
   - ROUGE (Recall-Oriented Understudy for Gisting Evaluation) evaluates the quality of text summarization or translation by comparing overlapping units such as n-grams, word sequences, and word pairs.
   - It is especially useful for assessing fluency and semantic alignment with reference translations.

3. **Exact Match Accuracy**:
   - This metric checks the percentage of predictions that exactly match the reference translations. It provides a strict evaluation of the model's ability to reproduce target sentences without deviations.


These metrics together provide a comprehensive evaluation of the translation model, balancing precision, recall, and semantic fidelity to ensure high-quality translations.


In [105]:
!pip install rouge

In [106]:
!pip install nltk

In [107]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(predictions, references):
    """
    Calculate BLEU scores for predicted and reference sentences.
    
    Args:
    - predictions (list): Model-generated sentences.
    - references (list): Ground truth (reference) sentences.
    
    Returns:
    - Average BLEU score.
    """
    smoothie = SmoothingFunction().method4
    scores = []
    for pred, ref in zip(predictions, references):
        score = sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie)
        scores.append(score)
    return sum(scores) / len(scores)




In [108]:
from rouge import Rouge

def calculate_rouge(predictions, references):
    """
    Calculate ROUGE scores for predicted and reference sentences.
    
    Args:
    - predictions (list): Model-generated sentences.
    - references (list): Ground truth (reference) sentences.
    
    Returns:
    - ROUGE scores as a dictionary.
    """
    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return scores



In [109]:
def calculate_exact_match_accuracy(predictions, references):
    """
    Calculate exact match accuracy between predictions and references.
    
    Args:
    - predictions (list): Model-generated sentences.
    - references (list): Ground truth (reference) sentences.
    
    Returns:
    - Exact match accuracy.
    """
    exact_matches = sum([1 for pred, ref in zip(predictions, references) if pred.strip() == ref.strip()])
    return exact_matches / len(references)




In [115]:
loss

0.2775474488735199

In [ ]:
import random

# Test setinin boyutunun yaklaşık yarısını seç
test_loader = list(test_loader)  # test_loader'ı bir listeye dönüştür
subset_size = len(test_loader) // 4  # Test setinin yarısı
subset_indices = random.sample(range(len(test_loader)), subset_size)  # Rastgele örnekle

# Rastgele alt kümeyi oluştur
subset_test_loader = [test_loader[i] for i in subset_indices]

predictions = []
references = []

# Alt küme üzerinde test döngüsü
for batch_num, batch in enumerate(subset_test_loader):
    eng_batch, tr_batch = batch
    for index, eng_sentence in enumerate(eng_batch):
        translation = translate(eng_sentence, device)
        print(translation)
        predictions.append(translation)
        references.append(tr_batch[index])

korkunç karşı ve annarchy.</s>
rs 1 lakh yaraları için rs 1 lakh için ört kazandı.</s>
polis bu sivil bu kişiye 13 kişiyi yanıtıldı.</s>
orada başk olmalı?</s>
hindistan, suniyet nant kumar, sushil kumar, sushil biri küçünü iki küçük kapı alıyor.</s>
bir tanrılar için ina benzerinde deniyle bir kazaları izin verileri için bir şekilde kadın bekleniyor.</s>
ancak bir anne sevdiğini söylemez.</s>
başka etti.</s>
hükümet cevaplar bunu cevaplar.</s>
kesinlik</s>
ama bu yanlış değil.</s>
bu mannet ve bu nedir?</s>
düşük olana birinden değildir.</s>
sorunuz sorunlarınız verilecek.</s>
onunla bir aile.</s>
sunni karar viee götürmek için karar verilmedi karar veriyor</s>
açarpışmam.</s>
6 atransından bir sunul için bir sunula veya 6- 6 spe seçenek bir sunulu'nda 6-s6 vidir</s>
oyuncular destekler hazır.</s>
uygulama/ yaşındı</s>
bir birlikte bekliydi.</s>
ata analisi ne itibarını nelerdir?</s>
konunun geliştirme yapmak için de hint olmadıkları bir şekilde bunun din duygulanmasını ifade etmeye b

In [122]:
predictions

['korkunç karşı ve annarchy.</s>',
 'rs 1 lakh yaraları için rs 1 lakh için ört kazandı.</s>',
 'polis bu sivil bu kişiye 13 kişiyi yanıtıldı.</s>',
 'orada başk olmalı?</s>',
 'hindistan, suniyet nant kumar, sushil kumar, sushil biri küçünü iki küçük kapı alıyor.</s>',
 'bir tanrılar için ina benzerinde deniyle bir kazaları izin verileri için bir şekilde kadın bekleniyor.</s>',
 'ancak bir anne sevdiğini söylemez.</s>',
 'başka etti.</s>',
 'hükümet cevaplar bunu cevaplar.</s>',
 'kesinlik</s>',
 'ama bu yanlış değil.</s>',
 'bu mannet ve bu nedir?</s>',
 'düşük olana birinden değildir.</s>',
 'sorunuz sorunlarınız verilecek.</s>',
 'onunla bir aile.</s>',
 'sunni karar viee götürmek için karar verilmedi karar veriyor</s>',
 'açarpışmam.</s>',
 "6 atransından bir sunul için bir sunula veya 6- 6 spe seçenek bir sunulu'nda 6-s6 vidir</s>",
 'oyuncular destekler hazır.</s>',
 'uygulama/ yaşındı</s>',
 'bir birlikte bekliydi.</s>',
 'ata analisi ne itibarını nelerdir?</s>',
 'konunun geli

In [123]:
predictions_new=[]
for predict in predictions:
    predictions_new.append(predict.split("</s>")[0])

In [124]:
# BLEU Score
bleu_score = calculate_bleu(predictions_new, references)
print(f"Average BLEU Score: {bleu_score}")
print("*********************")
# ROUGE Score
rouge_scores = calculate_rouge(predictions_new, references)
print(f"ROUGE Scores: {rouge_scores}")
print("*********************")

# Exact Match Accuracy
accuracy = calculate_exact_match_accuracy(predictions_new, references)
print(f"Exact Match Accuracy: {accuracy}")
print("*********************")


Average BLEU Score: 0.06362628192318476
*********************
ROUGE Scores: {'rouge-1': {'r': 0.27855274774048105, 'p': 0.2971562487522651, 'f': 0.2826997843268796}, 'rouge-2': {'r': 0.0776084009715923, 'p': 0.07821743641428855, 'f': 0.07657598317964263}, 'rouge-l': {'r': 0.2697391809758457, 'p': 0.2874223748526105, 'f': 0.2735727077568531}}
*********************
Exact Match Accuracy: 0.0206140350877193
*********************


In [125]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(200, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding)

In [126]:
optim

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)

In [127]:
import torch
# Modeli yükle
transformer = torch.load('MT_full_model.pth')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Modeli evaluation moduna al
transformer.eval()
def translate(eng_sentence,device):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence,device)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_turkish[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]
translation = translate("what should we do when the day starts?",device)
print(translation)

günlerin ne zaman başlamalıyız?</s>
